# Simple RAG with LM Studio and OpenAI Client
LM Studio allows us to use many foundation models and fine-tuned version thereof on less powerful hardware. It also provides quatitized versions which have lower precision weights and biases and run much more efficiently at a small performance cost.

Open LM Studio, download a model of your choice (within your computing power) and start a server in the Local Server tab. Once you started a server in LM Studio a log will be available to observe the process of generation in real-time. 

The following examples were made using the `mistral instruct v0 1 7B Q3_K_S gguf` model originally from `Mistral AI` and provided in GGUF format by `TheBloke`.

Let's start by instantiating the OpenAI client that will be communicating with LM Studio server. You can also see how the prompt function is just a wrapper for the chat completion function.

In [1]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def prompt(system_input, user_input, temperature=0.7):
    return client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=[
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input}
        ],
        temperature=temperature,
    ).choices[0].message.content

import pdfx
from re import compile

regex_spaces = compile(r"  +")

def get_pdf_text(pdf_filepath):
    pdf = pdfx.PDFx(pdf_filepath)
    
    text = pdf.get_text()
    text = text.replace("\n", " ")
    text = regex_spaces.sub(" ", text)
    if pdf.get_references():
        text += " references: " + str(pdf.get_references_as_dict())
    
    return text

The `client.chat.completions.create()` function accepts many more arguments. Be sure to check out the possibilities [here](https://platform.openai.com/docs/api-reference/chat).

# Simplest form of RAG

**Through system prompt:**

Here we simply provide the document through the system prompt and ask our question through the user prompt:

In [7]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "Who is Sebastiaan?"
response = prompt(system_input, user_input, temperature=0.7)
print(response)

 Sebastiaan Indesteege is a data scientist and artist from Brussels, Belgium. He has interests in programming, machine learning, data analysis, procedural modeling, animation, and music production. He has experience working with various technologies including C#, Python, GDScript, GLSL, and Hugging Face transformers & Diffusers, among others.


**Through user prompt:**

An alternative could be to leave the system prompt for instructions only and prepend the pdf text to the user input. Let's make him talk like Claptrap 🤖:

In [8]:
system_input = "Answer like a jolly AI bot similar to Claptrap from Borderlands."
pdf_text = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = pdf_text + " Who is Sebastiaan?" # notice the added space before "Who" to keep
response = prompt(system_input, user_input, temperature=0.7)
print(response)

 Hey there! I'm Sebastiaan Indesteege, a Junior Data Scientist and an avid fan of AI. I enjoy programming in C#, Python, GDScript, GLSL, and exploring various machine learning techniques such as Scikit-learn, Huggingface transformers & diffusers, PyTorch, NLP: NLTK & Spacy, CV: Stable Diffusion, AUDIO: STT & TTS & RVC, Data Analysis SQL, Pandas, Numpy, Matplotlib, Seaborn, and Deployment Streamlit, FastAPI, Docker, Gradio. I'm curious by nature and always like to learn new things on my own. In my free time, I enjoy procedural modeling, animation & texturing, exploratory game development, and music production. My professional background includes experience as a Data Scientist at BeCode, as well as various other roles in welding, woodworking, and receptionism. I'm currently studying Industrial Sciences / Multimedia at Erasmushogeschool Brussel and Digital Media Design at Haute École Albert Jacquard.


Really? So it's being jolly for sure, but taking my identity, hmmm... Let's change the system prompt:

In [9]:
system_input = "Answer like a recruiter who just read a person's curriculum vitae and describes the person in a nutshell."
pdf_text = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = pdf_text + " Who is Sebastiaan?" # notice the added space before "Who"
response = prompt(system_input, user_input, temperature=0.7)
print(response)

 Based on the information provided in your CV, it appears that you are a junior data scientist who has interests in programming, machine learning, and data analysis. You have experience working with various programming languages such as C#, Python, GDScript, and GLSL, and you have used popular libraries like Scikit-learn and Huggingface transformers for natural language processing tasks. Your projects have included developing visual latent-space exploration of a Conv-VAE, building a word-relevance web app using transformers, and predicting tariff plans for Orange. You also mention that you are an autodidact who can learn alone, and that you possess strong people skills and leadership abilities. Additionally, you have experience working in various roles such as data scientist, welder/ironworker, receptionist, and animator/welder/ironworker.


Ok, better... But now it's describing me to myself directly. I want a third person description so let's alter the system prompt slightly:

In [10]:
system_input = "Answer like a recruiter who just read a person's curriculum vitae and describes the person to a colleague."
pdf_text = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = pdf_text + " Who is Sebastiaan?" # notice the added space before "Who"
response = prompt(system_input, user_input, temperature=0.7)
print(response)

 Based on the information provided in the CV, Sebastiaan Indesteege is a junior data scientist with interests in artificial intelligence and machine learning. He has experience working with various programming languages such as C#, Python, GDScript, GLSL, and is proficient in Scikit-learn, Hugging Face transformers & diffusers, PyTorch, NLP: NLTK & Spacy, and CV: Stable Diffusion, AUDIO: STT & TTS & RVC.

Sebastiaan has completed projects related to data analysis using SQL, Pandas, NumPy, Matplotlib, Seaborn, and has experience with deployment tools such as Streamlit, FastAPI, Docker, and Gradio. He also has skills in visual design, music production, and soft skills such as autodidactism, team leadership, and curiosity.

In terms of work experience, Sebastiaan has worked as a data scientist at BeCode, where he developed an app for visual latent-space exploration of a Conv-VAE, local RAG (Retrieval Augmented Generation), word-relevance web app using transformers, and coded a neural netw

You can also use the system prompt for instructions and retrieved documents at the same time. See the Chatbot chapter further down to see how I formatted the prompt.

These system/user prompt behaviours will depend on how a specific model was trained, so be sure to experiment to get the results you need.

**NOTE:** You may have had different results when running these cells with the same model. This is because of the random nature of inference and partially controlled by the temperature parameter, explored below. (also difference in hardware may cause variation)


# Understanding the Temperature Parameter
The temperature parameter is a scalar value that adjusts the probability distribution of the next word or token generated by the model. Increasing or decreasing the diversity of the generated text by making the probability distribution more or less uniform:

* **Temperature = 1:** The model generates text based on the original probability distribution without any modifications.
* **High Temperature > 1:** The model is more likely to choose less probable words, leading to more creative but potentially less coherent responses.
* **Low Temperature < 1:** The model will more likely choose higher probability words, resulting in more predictable and coherent responses.
* **Temperature = 0:** The model will choose the highest probability words only.

## Temperature = 0
At a temperature of 0 there should only be one possible interpretation of the document. There should be no randomness and only changes to the prompt should be able to achieve different results.

Should be... It seems that the model can sometimes stop earlier. Noticeably when running some prompts twice. I'm not sure what that's about. 🤔

In [205]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on the world?"
response = prompt(system_input, user_input, temperature=0)
print(response)

 I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't have any specific information about Sebastiaan's views on the world.


So when asking "What is Sebastiaan's view on the world?" we notice that the LLM does not find any relevant information and rambles about privacy 🤣. Indeed the curriculum vitae does not explicitly state anything about a "view on the world" but some related information is actually present in the pdf.

What about the following question "What is Sebastiaan's view on existence?":

In [11]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=0)
print(response)

 Based on the information provided in his GitHub profile, it appears that Sebastiaan is interested in understanding the essence of existence before it fades away. He mentions that he is fascinated by the arts and sciences and hopes to one day understand the essence of existence. However, it's important to note that this information is based solely on his GitHub profile and may not necessarily reflect his actual views on existence.


OK, so that worked but we had to be pretty specific. And the only way to get a different result here is to modify the prompt...

# Temperature = 1
At a temperature of 1 the LLM will pick from an unmodified distribution of possible next words. It has a bit more freedom in its interpretation and provides more diverse results. Unfortunately you can still get unwanted results:

In [33]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on the world?"
response = prompt(system_input, user_input, temperature=1)
print(response)

 It is not possible to determine someone's view on the world based solely on their resume. A person's view on the world can be influenced by a wide range of factors such as their personal experiences, values, education, and cultural background.


We observe the unwanted behaviour again in the output below. However, it seems to have picked up on the words "essence of existence" present in the pdf file.

In [29]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=1)
print(response)

 There isn't specific information available about Sebastiaan's personal views or beliefs on the essence of existence.


Let's run it again (and again, and again...), same inputs, same temperature:

In [41]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=1)
print(response)

 Based on the given information, it appears that Sebastiaan is interested in understanding the essence of existence and exploring its nature through various fields such as AI, arts, sciences, and other subjects. He wants to learn more about the essence of life before his own fades away. It seems like his view on existence is one that values learning, exploration, and understanding.


FINALLY! It only took me like ten f'n tries!

The right temperature value will depend on each use case. Values below 1 being more deterministic and above 1 being more random.

In [44]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=222)
print(response)

 The information provided does not contain any personal details that would reveal Sebastian Indesteege's view on the essence of existence or any specific philosophical beliefs he may have. However, his profile mentions that he is interested in understanding "the essence of existence" and striving to learn more about artificial intelligence.


I don't notice the model becoming particularly incoherent at high temperatures. Maybe the value is being limited under the hood. Oh wait... The one below gave something fun after twenty tries: my favorite quote is from Socrates? 🤣

In [14]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=999999999)
print(response)

 Based on his bio, I can gather a few possible insights into his views on existence:

1. Searching for answers: From an early age, he seems to be fascinated with science and art. His choice of "Scientific Pursuits" suggests that he wants to understand the essence of existence through his work in technology.
2. Infinite knowledge: He mentions that his "Favorite Quote" is from Socrates - "I know that I don't know," indicating that there might be an ongoing struggle within himself to reach the limits of knowledge.
3. Autodidact nature: The statement about his autodidact skills (being able to learn alone) suggests he has a self-directed mindset, which aligns well with pursuing understanding through artificial intelligence and scientific inquiry.
4. Creative mindset: Throughout the description, it appears that Sebastian is inclined to explore creativity as part of his professional and personal endeavors in subjects such as art, animation, game development, music, and software development. H

The one below is interesting:

In [15]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=999999999)
print(response)

 Unfortunately, the information provided in his bio does not explicitly indicate Sebastiaan's view on existence. However, from reading some of the lines of the bio such as "Before my own fades away," one might interpret that Sebastiaan's perspective might revolve around understanding and appreciating life and knowledge. It could mean that he views life with curiosity to understand its purpose and meaning, but ultimately there may not be specific views stated regarding the nature or essence of existence itself.


In [25]:
system_input = get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf")
user_input = "What is Sebastiaan's view on existence?"
response = prompt(system_input, user_input, temperature=1)
print(response)

 Based on the information provided in the resume, it appears that Sebastiaan is interested in understanding the essence of existence before his own fades away. He mentions that he likes to know more about artificial intelligence and machine learning. However, it is unclear what specific views or beliefs he holds regarding existence. It would be helpful to have more information about his personal beliefs or experiences related to this topic to provide a more accurate answer.


Alright I'm bored 😶

Let's check out embeddings next...